# STEM Algorithm

## Step 1

In [1]:
import numpy as np
import pyomo.environ as pyo

#### Input

In [2]:
I, J = 2, 2

Cij = [
    [0.4, 0.3],
    [1, 0],
]
Aij = [
    [1, 1],
    [2, 1]
]
Bi = [400, 500]

In [3]:
f = lambda i, *x: np.dot(x, np.array(Cij)[i].reshape(-1, 1))

In [4]:
model = pyo.ConcreteModel()

model.i = pyo.RangeSet(0, I-1)
model.j = pyo.RangeSet(0, J-1)

model.a = pyo.Param(model.i, model.j, initialize=dict(np.ndenumerate(Aij)))
model.b = pyo.Param(model.i, initialize=dict(enumerate(Bi)))
model.c = pyo.Param(model.i, model.j, initialize=dict(np.ndenumerate(Cij)))

model.x = pyo.Var(model.j, domain=pyo.NonNegativeReals)


def cons_rule(model, i):
    return sum(model.a[i, j]*model.x[j] for j in model.j) <= model.b[i]
model.cons = pyo.Constraint(model.i, rule=cons_rule)

In [5]:
income_matrix = np.empty((I, I))

In [6]:
for i in range(I):
    model.obj = pyo.Objective(
        expr=sum(model.c[i, j]*model.x[j] for j in model.j),
        sense=pyo.maximize
    )

    opt = pyo.SolverFactory('cplex')
    opt.solve(model)

    x = [pyo.value(model.x[j]) for j in model.j]
    obj = pyo.value(model.obj)
    print(obj)
    
    del model.obj

    matrix = np.array([])
    for j in range(I):
        if i == j:
            income_matrix[i, j] = obj
            continue  
        income_matrix[i, j] = f(j, *x)

130.0
250.0


In [7]:
alpha = np.empty(I)

In [8]:
for i in range(I):
    f_star = income_matrix[i, i]
    f_min = min(income_matrix[:, i])
    c = np.floor(1/np.sqrt(sum(np.array(Cij)[i]**2)))

    if income_matrix[i, i] > 0:
        alpha[i] = (f_star-f_min)*c/(f_star)
    else:
        alpha[i] = (f_min-f_star)*c/(f_min)


In [9]:
pi = alpha/sum(alpha)

In [10]:
model.pi = pyo.Param(model.i, initialize=pi, mutable=True)
model.lambd = pyo.Var(domain=pyo.NonNegativeReals)
model.obj = pyo.Objective(expr=model.lambd)

model.cons_dm = pyo.ConstraintList()
for i in model.i:
    model.cons_dm.add(
        expr=model.lambd>=(income_matrix[i, i]-sum(model.c[i, j]*model.x[j] for j in model.j))*model.pi[i]
    )

In [11]:
opt = pyo.SolverFactory('cplex')
opt.solve(model)

x = [pyo.value(model.x[j]) for j in model.j]
res = [f(i, *x) for i in range(I)]

for func in res:
    print(func[0])

104.0
230.0


## Step 2

#### Input

In [12]:
func_old = np.array([104, 230])
adjustment_data = {1: 30}

In [13]:
adjustment_key = list(adjustment_data.keys())[0]
adjustment_value = adjustment_data[adjustment_key]

for i in range(I):
    model.pi[i] = 1/(I-1)
model.pi[adjustment_key] = 0

for i in range(I):
    if i == adjustment_key:
        model.cons_dm.add(
            expr=sum(model.c[i, j]*model.x[j] for j in model.j)>=(func_old[i]-adjustment_value)
        )
        continue
    model.cons_dm.add(
        expr=sum(model.c[i, j]*model.x[j] for j in model.j)>=func_old[i]
    )

In [14]:
opt = pyo.SolverFactory('cplex')
opt.solve(model)

x = [pyo.value(model.x[j]) for j in model.j]
res = [f(i, *x) for i in range(I)]

for func in res:
    print(func[0])

110.0
200.0
